## Parameter Tuning with SciOpt : MNIST
=============

Task : 

1. Construct a classification model on the MNIST data set in the form of a deep neural network of your choice.

2. Perform and compare two or more methods of hyperparameter optimization on this model, and comment on the comparison.




### Task 2 :  Perform and compare  hyperparameter optimization

### 4.1 Grid Search : 350 years
- Because of the exhaustiveness Grid Search was not a feasible option at all for our case.
- Our case below : 137 million possible parameter combination set. (4*7*3*6*5*3*65*140*2)
- Total time taken @ 80s for 1 parameter set : 349.04 years (4*7*3*6*5*3*65*140*2)/(60.0*60.0*24.0*365) * 80.0 # years-
- PS : 80s training duration as observed in training of one parameter set with fitting function for Scikit-Optimise.


### 4.2 Random Search : 
**Performance** :In our case, the scikit-optimise library' dummy  random optimise in a 4 core 2.5GHz CPU with 6 GB RAM, ran for 20 mins and yielded 98.0% accuracy which is above the manually  implemented 96.7% accuracy.
**Basic Info**: It dramatically reduce search space, but the probability of getting to the globally optimal hyperparameter reduces to almost zero  with addition of each hyperparameter. 
- e.g What is the probability of getting the 1 best hyperparameters set from 137,592,000  parameter set = 1 / 137,592,000
However it is more effective in practice.
- **Cons** : The biggest cons of the random search is that when it find the approximate optimal hyperparameter values, it is not able to gain cues from it,  to further optimise the parameters.
- **Pros** : However one of the  advantage is that because  other optimisation tehcniqques are so expensiveness  in terms of time and computation, when no  good default hyper parameter estimate is avaialable, it can be used to find the approximate good default hyperparameter configuration set. This hyperparameter then can be used as the default parameter setting for later Bayesian optmisation and  automated optimisation algorithms.

### 4.3 Bayesian Optimization
**Performance** : In our case, the scikit-optimise library's Bayesian optimization algorithm in a 4 core 2.5GHz CPU with 6 GB RAM, ran for 1.5 hours and 98.2% accuracy which is above the manually  implemented 96.7% accuracy and 97.6% accuraccy of evolutionary algorithm.

**Implementation**:In this project, we  used scikit-optimise library's Bayesian optimization (Gaussian process) because of its compatibility with  tensorflow, keras making it suitable for future scalable , massively parallelised hyper-parameter optimisation approach.
For cases of neural networks with large no of  hyper-parameters and where function evaluation is the mean  cross-validated score across multiple folds, gaussian prior is much quicker to evaluate, leading to faster hyperparameter next set choice. And hence we have used the Bayesian optimisation with Gaussian process.

** Basic Info **
- Bayesian optimization  is based on Bayes theorem. Bayes theorem uses the prior probability i.e what we know  already to deduce the posterior probability. 
- Correlating this  with the HyperParameter optimization, the next best hyper-parameter set are the unknowns, which are then determined by what we know already i.e earlier iterations of hyper-parameter configuration and the  evaluation(accuracy in our case). 
- Bayesian optimization as such is well suited for functions that are very expensive to evaluate. In our particular case, we observed that  tuning just 9 hyper-parameters meant that we had 140 million hyper-parameter set which is very very exhaustive and expensive.
-  They reduce the expensive operation by trading off exploration for exploitation so as to minimize the no. of  queries. Number of acquisition functions such as Probability of improvement, Bayessian Expected Loss(EL), upper/lower confidence bounds, Thompson sampling or mix of these exists. These are the functions that we try to minimize with each parameter iteration.

Other options that exists are,
1. BayesSearchCV - similar to grid search but  more efficient in that only a fixed number of parameter set is sampled from specific distribution.
2. dummy_minimize - random search by uniform sampling
3. forest_minimize -
4. gbrt_minimize : used for highly expensive evaluation function.

### 4.4 Evolutionary Optimisation :
**Performacce** : In our case, the genetic algorithm in a 4 core 2.5GHz CPU with 6 GB RAM, ran for 6 hours and yielded the Convolution neural network with following architecture with 97.7% accuracy which is slightly above the manually  implemented 96.7% accuracy. The fact that we clipped the hyperparameters(nodes specifically) and evolution parameters( child population and the generation) may have been influential to not reach state of art performance.
**Implementation** : Please see file "MNIST with Genetic Algorithm.html" for its detailed implementation

**Basic Info**
The evolutionary optimisation use the biological evolution concept inspired optimisation for hyperparameters. 
The general evolution concept works  on a parent child architecture as
1. Parents (1st generation) reproduce i.e generate n initial population(random tuples of hyperparameters in our case)
2. Parent evaluate fitness of each child (fitness function)
3. Select best fit child for reproduction (Parents )
4. Generate next generation childs with breeding(cross over, mutation of best hyper parameters of 1st generation or leading parent)
5. Evaluate and replace least fit with new child (low performing hyperparam replacement)
6. Repeat 3 through 5, until stopping criteria(generation elapses, target eval reached)




## Optimiser Performance Comment

Both of the optimiser's performance are well below the state of art performance of above 99.5%. This should be considered in the light that we provided limited search space because of the limited run time.
The next interesting aspect would be to  provide much richer search space for both algorithms at the expense of more training time. 


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

# Keras Imports
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input, Reshape, MaxPooling2D, Conv2D, Dense, Flatten,Dropout 
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.python.keras.models import load_model

# SkOpt Bayesian Hyperparameter optimisation imports
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
#from skopt.plots import plot_convergence
#from skopt.plots import plot_objective, plot_evaluations
#from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

# 1 :  Data  Load

MNIST Data as downloaded from  tensorflow

In [5]:
# Change Load Data format
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# Reshape to  this
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

# One hot Encoding
data.test.cls = np.argmax(data.test.labels, axis=1)
validation_data = (data.validation.images, data.validation.labels)


Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [7]:
img_size = 28 # We know that MNIST images are 28 pixels in each dimension.
num_channels = 1 # greyscale images hence 1 channel
num_classes = 10 # for 10 digits from 0 to 9

# 2 Keras ConvNet  

###  2 (Conv  Layer + MaxPool + Dropout ) + 2 Fc  Layer

Create a simple keras convnet MNIST digit classification model

In [8]:
dropout_values = 0.2
activation = 'relu'
batch_sizes = 20

# Model Creation
model = Sequential()
model.add(InputLayer(input_shape=(img_size*img_size,))) # accepts flattened images    
# 784 to  (28, 28, 1) Conv layers expected shape
model.add(Reshape((img_size, img_size, 1)))

model.add(Conv2D(kernel_size=5, strides=1, filters=36, padding='same',activation=activation, name='layer_conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(dropout_values))
model.add(Conv2D(kernel_size=5, strides=1, filters=36, padding='same',activation=activation, name='layer_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(dropout_values))

# Flatten  to feed to fully connected layer i.e dense layer in keras
model.add(Flatten())    
model.add(Dense(64,activation='relu',name='fc_layer_0'))
model.add(Dropout(dropout_values))
model.add(Dense(num_classes, activation='softmax'))
optimizer = Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 

convnet = model.fit(x=data.train.images,  y=data.train.labels,  epochs=1,
                        batch_size=batch_sizes, validation_data=validation_data)#,callbacks=[callback_log])
accuracy = convnet.history['val_acc'][-1]
print('ConvNet model accuracy is : ',accuracy)

Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 179s 3ms/step - loss: 0.3207 - acc: 0.9032 - val_loss: 0.1096 - val_acc: 0.9672
ConvNet model accuracy is :  0.9671999945640564


# 3 HyperParameter Optimisation



## 3.1 Important Hyperparameters Identification

Not all hyper parameters have equal significance. And since with each addition of a hyperparameter, the hyperparameter search space grows exponentially, it would be computationally and time exhaustive  to search through all the hyper parameters available. 

Hence it is much more better to  identify important  hyperparameters, to reduce the search space.

1. Learning rate
2. Conv Layers
3. Patch size
4. max_pool size
5. hidden layers
6. hidden nodes
7. batch
8. optimiser
9. dropout values

While there are other important hyperparameters too, we will limit ourselves to these for this project

### 3.2 HyperParameter Boundary

Define HyperParameter boundary to search within. It is not feassible to search in the infinite boundary space.

In [72]:
# we will search over k only for 1e-k i.e 0.01, 0.001, 0.0001, 0.00001 , because we specified log-uniform
dim_learning_rate = Real(low=1e-5, high=1e-2, prior='log-uniform',  name='learning_rate') 
dim_dropout_values = Real(low=0.1, high=0.7 , prior='uniform', name='dropout_values') 
dim_num_conv_layers = Integer(low=2, high=4, name='num_conv_layers')
dim_patch_sizes = Integer(low=2, high=7, name='patch_sizes')
dim_max_pool_sizes = Integer(low=1, high=4, name='max_pool_sizes')
dim_num_dense_layers = Integer(low=2, high=5, name='num_dense_layers')
dim_num_hidden_nodes = Integer(low=64, high=1280, name='num_hidden_nodes')
dim_batch_sizes = Integer(low=10, high=150, name='batch_sizes')
dim_optimisers = Categorical(categories=['Adam','SGD'],    name='optimisers')
#dim_activation = Categorical(categories=['relu', 'sigmoid'],    name='activation')

# Combine boundaries, to be fed as single boundary list
dimensions = [dim_learning_rate,dim_num_dense_layers, dim_num_hidden_nodes, dim_batch_sizes, dim_optimisers,\
              dim_dropout_values, dim_num_conv_layers, dim_patch_sizes, dim_max_pool_sizes ]



### 3.3 Default parameter 

While the hyperparameter optimiser runs for the first time, it runs the fitness function with the default parameters. 

If good default hyperparameter values are provided, then it will significantly  help reach to global optimal hyperparameter configuration.

Hence it is always a good idea to set default parameters to good parameter values, after literature study, whenever feasible

In [73]:
# Start search from a decent choice, so that the search narrows faster
default_parameters = [0.001, 2, 64, 20,'Adam',0.1, 2, 5,2]

### 3.4 Create Model

We now create model that will accept the hyperparameters configuration set, create model and return it

In [32]:
#Create a keras model
def create_model(learning_rate, num_dense_layers,  num_hidden_nodes,  optimisers, dropout_values ,\
                num_conv_layers, patch_sizes, max_pool_sizes ):
    activation = 'relu'
    model = Sequential()
    model.add(InputLayer(input_shape=(img_size*img_size,))) # accepts flattened images    
    # 784 to  (28, 28, 1) Conv layers expected shape
    model.add(Reshape((img_size, img_size, 1)))
    
    for j in range(num_conv_layers):           
        model.add(Conv2D(kernel_size=5, strides=1, filters=16, padding='same', activation=activation, \
                         name='layer_conv{0}'.format(j+1) ))
        model.add(MaxPooling2D(pool_size=(2,2), strides=2)) #maxpool by 2x2 window. 2x2 will halve the space
        model.add(Dropout(dropout_values))
    # Flatten  to feed to fully connected layer i.e dense layer in keras
    model.add(Flatten())    
    for i in range(num_dense_layers):        
        model.add(Dense(num_hidden_nodes,activation=activation,name='fc_layer_{0}'.format(i+1)))
        model.add(Dropout(dropout_values))

    model.add(Dense(num_classes, activation='softmax'))
    optimizer = eval(optimisers)(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])    
    return model

### 3.5 Fitness Function

Now we create the fitness function. This function accepts each parmeter set configuration, creates model, fits it and performs evaluation.

In [76]:
from datetime import datetime as dt
from skopt import dump, load
save_best_model_path = 'best_conv_model'
best_accuracy = 0.0

# optimiser function. skopt performs minimisation instead of maximisation, hence we negate the accuracy
# @use_named_args  used so that the function can be called as a single list as required by skopt i.e fitness(x=[1e-4, 3, 256, 'relu'])
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers,  num_hidden_nodes, batch_sizes, optimisers, dropout_values ,\
           num_conv_layers, patch_sizes, max_pool_sizes):
    # Print the hyper-parameters.
    print('learning rate : ', learning_rate,'. Num_dense_layers:', num_dense_layers,'. num_hidden_nodes:', num_hidden_nodes)
    print('optimisers: ', optimisers,'.  batch_sizes: ',batch_sizes,'. dropout_values : ',dropout_values,\
          'num_conv_layers =', num_conv_layers,'. patch_sizes = ',patch_sizes,'. max_pool_sizes =', max_pool_sizes)    
        
    # Create the neural network with the new hyper-parameters.
    model = create_model(learning_rate=learning_rate,           num_dense_layers=num_dense_layers,\
                         num_hidden_nodes=num_hidden_nodes,  optimisers=optimisers, dropout_values = dropout_values,\
                        num_conv_layers = num_conv_layers, patch_sizes = patch_sizes, max_pool_sizes = max_pool_sizes )   
    history = model.fit(x=data.train.images,  y=data.train.labels,  epochs=1,
                        batch_size=batch_sizes, validation_data=validation_data)#,callbacks=[callback_log])
    # Get the classification accuracy on the validation-set after the last training-epoch.
    accuracy = history.history['val_acc'][-1]
    print("Accuracy: {0:.2%}".format(accuracy),)
    global best_accuracy # global to preserve  the accuracy beyond this function scope

    if accuracy > best_accuracy:
        model.save(save_best_model_path)
        best_accuracy = accuracy    
    del model # free memory for next model, once performance evaluation with current hyper parameter is done.
    
    K.clear_session() # reset graph, other wise model will be added to same tensorflow graph everytime with new hyperparams
    return -accuracy # scikit opt does minimisation instead of maximisation, hence  we negate it

### 3.6 Test & Debug fitness function

Because the fitness function may  contain error, we should test it and debug it prior to running the optimisation across full hyperpareters

In [13]:
# Sample check with default parameter
fitness(x=default_parameters)

learning rate :  0.001 . Num_dense_layers: 2 . num_hidden_nodes: 64
optimisers:  Adam .  batch_sizes:  20 . dropout_values :  0.1 num_conv_layers = 2 . patch_sizes =  5 . max_pool_sizes = 2
Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 87s 2ms/step - loss: 0.2310 - acc: 0.9279 - val_loss: 0.0664 - val_acc: 0.9820
Accuracy: 98.20%


-0.9819999964237213

### 3.7 Run HyperParameter Optimisation

Now we run the hyperparameter optimisation using scikit-optimise's  Bayesian Optimisation with Gaussian Regression

In [77]:
# HyperPArameter Optimisation
hp_search_result = gp_minimize(func=fitness,    # function to minimise
                            dimensions=dimensions, # hyperparamter search boundary to search through
                            acq_func='EI', # Expected Improvement.
                            n_calls=20,   #40 no of calls to func to find minimum
                            x0=default_parameters
                           ,n_jobs = 4 # number of cores to run in parallel
                           )


learning rate :  0.001 . Num_dense_layers: 2 . num_hidden_nodes: 64
optimisers:  Adam .  batch_sizes:  20 . dropout_values :  0.1 num_conv_layers = 2 . patch_sizes =  5 . max_pool_sizes = 2
Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 77s 1ms/step - loss: 0.2257 - acc: 0.9293 - val_loss: 0.0567 - val_acc: 0.9812
Accuracy: 98.12%
learning rate :  1.2273662556515456e-05 . Num_dense_layers: 5 . num_hidden_nodes: 101
optimisers:  SGD .  batch_sizes:  100 . dropout_values :  0.40201642903982104 num_conv_layers = 3 . patch_sizes =  6 . max_pool_sizes = 3
Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 63s 1ms/step - loss: 2.3416 - acc: 0.0989 - val_loss: 2.3029 - val_acc: 0.0948
Accuracy: 9.48%
learning rate :  0.0026281154224798995 . Num_dense_layers: 2 . num_hidden_nodes: 679
optimisers:  SGD .  batch_sizes:  83 . dropout_values :  0.5902048504250774 num_conv_layers = 3 . 

# 4 Results

Now we inspect best hyperparameter set and its performance.

We also inspect other hyperparameter configuration set and  thier corresponding performance.

In [78]:
# Best HyperParameters
print('Best hyperparameter is :', hp_search_result.x )
print('Best accuracy is : ', hp_search_result.fun )
print(' ')
print('All hyper params and their respective accuracy is')
sorted(zip((hp_search_result.func_vals*-100).astype(np.float16), hp_search_result.x_iters), reverse=True)


Best hyperparameter is : [0.0031902961765478046, 3, 1071, 150, 'Adam', 0.11920412766761226, 2, 7, 2]
Best accuracy is :  -0.9822000133991241
 
All hyper params and their respective accuracy is


[(98.25,
  [0.0031902961765478046, 3, 1071, 150, 'Adam', 0.11920412766761226, 2, 7, 2]),
 (98.1, [0.001, 2, 64, 20, 'Adam', 0.1, 2, 5, 2]),
 (97.5, [0.01, 2, 64, 150, 'Adam', 0.1, 2, 7, 1]),
 (97.25, [0.01, 3, 64, 150, 'Adam', 0.35092620217950443, 2, 3, 2]),
 (97.2,
  [0.0009187946785502826, 3, 243, 10, 'Adam', 0.36761659600249386, 3, 6, 2]),
 (96.9,
  [0.00022921001012124313, 3, 280, 81, 'Adam', 0.10331790957943952, 2, 6, 3]),
 (95.2, [0.01, 2, 64, 10, 'Adam', 0.1, 2, 2, 1]),
 (94.9,
  [0.00025857017175947894, 3, 493, 38, 'Adam', 0.4060449389435037, 3, 6, 3]),
 (83.56,
  [0.0004440020004428979, 3, 532, 119, 'Adam', 0.5471514032474885, 3, 3, 4]),
 (82.25, [1e-05, 2, 64, 10, 'Adam', 0.1, 2, 6, 3]),
 (43.6, [1e-05, 2, 64, 10, 'Adam', 0.2277791146830224, 3, 7, 2]),
 (42.97,
  [0.00062003703251284, 4, 499, 97, 'Adam', 0.6647005975141239, 2, 3, 3]),
 (16.48,
  [6.821654549728117e-05, 4, 1148, 101, 'Adam', 0.4941064065027715, 3, 6, 4]),
 (15.48,
  [0.0022637413889353745, 5, 838, 125, 'SGD', 

## Random Grid Search CV

Now, we implement the Random Grid Search to see its  performance.

In [83]:
from skopt import dummy_minimize
random_grid_search = dummy_minimize(func=fitness,    # function to minimise
                            dimensions=dimensions, # hyperparamter search boundary to search through                            
                            n_calls=20   #20 no of calls to func to find minimum                            
                           )


learning rate :  2.5959833184971615e-05 . Num_dense_layers: 3 . num_hidden_nodes: 828
optimisers:  SGD .  batch_sizes:  89 . dropout_values :  0.6611469037516559 num_conv_layers = 3 . patch_sizes =  3 . max_pool_sizes = 4
Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 86s 2ms/step - loss: 2.7014 - acc: 0.0999 - val_loss: 2.3027 - val_acc: 0.1310
Accuracy: 13.10%
learning rate :  0.0006400437068443558 . Num_dense_layers: 4 . num_hidden_nodes: 101
optimisers:  SGD .  batch_sizes:  114 . dropout_values :  0.16060957932136827 num_conv_layers = 4 . patch_sizes =  5 . max_pool_sizes = 4
Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 72s 1ms/step - loss: 2.3030 - acc: 0.0955 - val_loss: 2.3027 - val_acc: 0.0828
Accuracy: 8.28%
learning rate :  0.0008775168799193001 . Num_dense_layers: 2 . num_hidden_nodes: 891
optimisers:  Adam .  batch_sizes:  112 . dropout_values :  0.140371

Train on 55000 samples, validate on 5000 samples
Epoch 1/1
55000/55000 [==============================] - 79s 1ms/step - loss: 2.6613 - acc: 0.0964 - val_loss: 2.3019 - val_acc: 0.0990
Accuracy: 9.90%


In [84]:
print('Best accuracy with random grid  search is   : ',random_grid_search.fun)
print('Best accuracy with Bayesian Optimisation is : ', hp_search_result.fun )

Best accuracy with random grid  search is   :  -0.9807999932289123
Best accuracy with Bayesian Optimisation is :  -0.9822000133991241
